In [1]:
import numpy as np
import pandas as pd
import yfinance as yf

In [2]:
df = yf.download('AAPL').reset_index()

[*********************100%%**********************]  1 of 1 completed


In [3]:
def get_macd(df, is_daily, is_stock):
    if is_daily:
        factor = 1
        col_prefix = 'daily'
    else:
        factor = 5 if is_stock else 7
        col_prefix = 'weekly'
    # Obtain the moving averages for the MACD
    slow_ma = df['Close'].ewm(span=factor*26, adjust=False).mean()
    fast_ma = df['Close'].ewm(span=factor*12, adjust=False).mean()
    # Calculate the MACD and then use that to generate the signal line
    
    df.loc[:, col_prefix + '_macd'] = fast_ma - slow_ma
    df.loc[:, col_prefix + '_signal'] = (
        (fast_ma - slow_ma)
        .ewm(span=factor*9, adjust = False)
        .mean()
    )

    return df.dropna()

In [4]:
df = get_macd(df, is_daily=True, is_stock=True)
df = get_macd(df, is_daily=False, is_stock=True)

In [5]:
df.to_csv('output.csv')